In [101]:
import urllib.request
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
urllib.request.urlretrieve(url, 'car.data')


('car.data', <http.client.HTTPMessage at 0x2156919d0d0>)

In [102]:
import numpy as np
data = np.loadtxt('car.data', dtype="str", delimiter=',')
X, y = data[:,:-1], data[:,-1]
X.shape, y.shape

((1728, 6), (1728,))

In [103]:
combinacoes = 1
for i in range(X.shape[1]):
    valores = set(X[:,i])
    combinacoes *= len(valores)
    print(valores)
print(combinacoes)


{'high', 'low', 'vhigh', 'med'}
{'high', 'low', 'vhigh', 'med'}
{'2', '5more', '3', '4'}
{'2', 'more', '4'}
{'big', 'small', 'med'}
{'high', 'low', 'med'}
1728


In [104]:
#algoritmo ZeroR
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter

def maisFrequente(y):
    return Counter(y.flat).most_common(1)[0][0]
class ZeroR():
    def fit(self, X, y):
        self.resposta = maisFrequente(y)
    def predict(self, X):
        y = np.empty((X.shape[0]), dtype='<U5')
        y[:] = self.resposta
        return y
modelo = ZeroR()
modelo.fit(X,y)
ypred = modelo.predict(X)
accuracy_score(y, ypred), (ypred == y)


(0.7002314814814815, array([ True,  True,  True, ...,  True, False, False]))

In [105]:
def impureza(y):
    labels = list(set(y))
    labels.sort()
    probabilidades = np.zeros((len(labels),))
    for i,k in enumerate(labels):
        probabilidades[i] = sum(y==k)/len(y)
    result = 1 - sum(probabilidades**2)
    return result

ytmp = y[:]
valor = impureza(ytmp)
valor 

0.457283763074417

In [106]:
# calculando a impureza de cada característica
def impurezaValor(x, y, valor):
    iguais = x==valor
    impurezaIguais = impureza(y[iguais])
    propIguais = sum(iguais)/len(y)
    impurezaDiferentes = impureza(y[~iguais])
    propDiferentes = sum(~iguais)/len(y)
    impurezaTotal = propIguais*impurezaIguais + propDiferentes*impurezaDiferentes
    return impurezaTotal
impurezaValor(X[:,0],y,'vhigh')
    

0.44934645776177407

In [107]:
# função def para encontrar a impureza minima

def impurezaMinima(X, y):
    impurezas = []
    caracteristicasValores = []
    for i in range(X.shape[1]):
        valores = sorted(list(set(X[:,i])))
        for valor in valores:
            caracteristicasValores.append([i,valor])
            impurezaValorCaracteristica = impurezaValor(X[:,i], y, valor)
            impurezas.append(impurezaValorCaracteristica)
    caracteristicasValores = np.array(caracteristicasValores)
    impurezas = np.array(impurezas)
    menorImpureza = np.argmin(impurezas)
    caracteristica, valor = caracteristicasValores[menorImpureza]
    return impurezas[menorImpureza], int(caracteristica), valor
impurezaMinima(X, y) 


(0.38615712609310704, 3, '2')

In [108]:
#arvore de decisões em que os atributos são discretos, logo é um problema de classificação.
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter
from sklearn.base import BaseEstimator, ClassifierMixin

def maisFrequente(y):
    return Counter(y.flat).most_common(1)[0][0]
class Arvore(BaseEstimator, ClassifierMixin):
    def fit(self, X, y):
        self.impureza,self.caracteristica,self.valor = impurezaMinima(X,y)
        iguais = X[:,self.caracteristica] == self.valor
        if sum(iguais)>0 and sum(~iguais)>0:
            self.iguais = Arvore()
            self.iguais.fit(X[iguais,:],y[iguais])
            self.diferentes = Arvore()
            self.diferentes.fit(X[~iguais,:],y[~iguais])
        else:
            self.resposta = maisFrequente(y)
    def predict(self, X):
        y = np.empty((X.shape[0]), dtype='<U5')
        if hasattr(self, 'resposta'):
            y[:] = self.resposta
        else:
            iguais = X[:,self.caracteristica] == self.valor
            y[iguais] = self.iguais.predict(X[iguais,:])
            y[~iguais] = self.iguais.predict(X[~iguais,:])

        return y
modelo = Arvore()
modelo.fit(X,y)
ypred = modelo.predict(X)
accuracy_score(y, ypred), (ypred == y)

(0.7002314814814815, array([ True,  True,  True, ...,  True, False, False]))

In [109]:
# usar cross validate e Mae para calcular o score caso o valor do array seja 100%. Array com 100% de acuracia implica em erro na hora produção.
# nesse caso nao precisaria do fazer a validação cruzada.
from sklearn.model_selection import cross_validate

scores = cross_validate(Arvore(), X, y)
scores['test_score'], np.mean(scores['test_score'])

(array([0.69942197, 0.69942197, 0.69942197, 0.70144928, 0.70144928]),
 0.7002328893356791)

Atributos continuos # ValueError: Found input variables with inconsistent numbers of samples: [150, 2] 



In [110]:
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data[:2:], iris.target



In [ ]:
#algoritmo ZeroR
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter

def maisFrequente(y):
    return Counter(y.flat).most_common(1)[0][0]
class ZeroR():
    def fit(self, X, y):
        self.resposta = maisFrequente(y)
    def predict(self, X):
        y = np.empty((X.shape[0]))
        y[:] = self.resposta
        return y
modelo = ZeroR()
modelo.fit(X,y)
ypred = modelo.predict(X)
accuracy_score(y, ypred), (ypred == y)

In [ ]:
#arvore de decisões em que os atributos são contínuos (numéricos), logo é um problema de Regressão.
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter
from sklearn.base import BaseEstimator, ClassifierMixin

def maisFrequente(y):
    return Counter(y.flat).most_common(1)[0][0]
class Arvore(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.caracteristica = 2
    def fit(self, X, y):
        self.valor = np.mean(X[:,self.caracteristica])
        maiores = X[:,self.caracteristica] > self.valor
        if sum(maiores)>0 and sum(~maiores)>0:
            self.maiores = Arvore()
            self.maiores.fit(X[maiores,:],y[maiores])
            self.menores = Arvore()
            self.menores.fit(X[~maiores,:],y[~maiores])
        else:
            self.resposta = maisFrequente(y)
    def predict(self, X):
        y = np.empty((X.shape[0]))
        if hasattr(self, 'resposta'):
            y[:] = self.resposta
        else:
            maiores = X[:,self.caracteristica] > self.valor
            y[maiores] = self.maiores.predict(X[maiores,:])
            y[~maiores] = self.maiores.predict(X[~maiores,:])

        return y
modelo = Arvore()
modelo.fit(X,y)
ypred = modelo.predict(X)
accuracy_score(y, ypred), (ypred == y)

In [ ]:
#vizualizar as regiões de decisão
import matplotlib.pyplot as plt

def plotDecisacao(modelo, X, y):
    modelo.fit(X, y)
    x0s = np.linspace(np.min(X[:,0])-0.2, np.max(X[:,0])+0.2, 100)
    x1s = np.linspace(np.min(X[:,1])-0.2, np.max(X[:,1])+0.2, 100)
    x0, x1 = np.meshgrid(x0s, x1s)
    Xdec = np.c_[x0.ravel(), x1.ravel()]
    ypred = modelo.predict(Xdec)
    plt.contourf(x0, x1, ypred.reshape(x0.shape), alpha=0.25)
    for k in set(y): 
        plt.plot(X[:,0][y==k], X[:,1][y==k], 'o', label=f"{iris.target_names[k]} [{k}]")
    plt.xlabel(f"X[0]: {iris.feature_names[0]}")
    plt.ylabel(f"X[1]: {iris.feature_names[1]}")
    plt.show() 
plotDecisacao(Arvore(), X, y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
plotDecisacao(DecisionTreeClassifier(), X, y)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
plotDecisacao(KNeighborsClassifier(), X, y)

In [ ]:
from sklearn.linear_model import LogisticRegression
plotDecisacao(LogisticRegression(), X, y)